In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import ttest_ind
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

ModuleNotFoundError: No module named 'scipy'

In [ ]:
# Reading variation table and web_data table
df4 = pd.read_csv("df_final_experiment_clients.csv")
merged_df = pd.read_csv('merged_df_final_web_data.csv')

#Merging to merged_df_final
merged_df_final = pd.merge(merged_df, df4, on='client_id', how='inner')

#converting to a csv file
merged_df_final.to_csv('merged_df_final.csv', index=False)

#Reading df1
df1 = pd.read_csv("df_final_demo.csv")

# Merging customer_info with merged_df_final
final_v1 = pd.merge(merged_df_final, df1, on='client_id', how='inner')



In [ ]:
final_v1.isnull().sum()
final_v1=final_v1.dropna()


In [ ]:
final_v1.to_csv('final_v1.csv', index=False)

In [ ]:
df_control = final_v1[final_v1['Variation'] == 'Control']

In [ ]:
df_control

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
13,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:03,Control,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0
14,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:29:01,Control,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0
15,4033851,762728880_76361333336,949661017_22392791362_127391,confirm,2017-04-05 12:28:52,Control,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0
16,4033851,762728880_76361333336,949661017_22392791362_127391,step_3,2017-04-05 12:26:08,Control,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0
17,4033851,762728880_76361333336,949661017_22392791362_127391,step_2,2017-04-05 12:24:43,Control,12.0,149.0,63.5,M,2.0,142642.26,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417643,433098,5505424_50616523266,773565250_67577573147_389841,start,2017-05-13 15:30:47,Control,6.0,73.0,25.0,U,2.0,34419.62,2.0,5.0
417651,2908510,814969699_90652851448,562606085_36368381773_92090,start,2017-05-10 22:57:17,Control,21.0,252.0,34.0,M,3.0,141808.05,6.0,9.0
417652,2908510,814969699_90652851448,562606085_36368381773_92090,step_2,2017-05-10 22:56:31,Control,21.0,252.0,34.0,M,3.0,141808.05,6.0,9.0
417653,2908510,814969699_90652851448,562606085_36368381773_92090,step_1,2017-05-10 22:56:23,Control,21.0,252.0,34.0,M,3.0,141808.05,6.0,9.0


In [ ]:
df_control['date_time'] = pd.to_datetime(df_control['date_time'])

# Sort the DataFrame by client_id, visitor_id, and date_time
df_control = df_control.sort_values(by=['client_id', 'visitor_id', 'date_time'])

# Initialize lists to store new dataframe data
new_data = []

# Dictionary to store start times, end times, and last step
client_data = {}

# Iterate over rows to collect start and confirm times, last step, and calculate other needed data
for index, row in df_control.iterrows():
    client_id = row['client_id']
    visitor_id = row['visitor_id']
    
    if (client_id, visitor_id) not in client_data:
        client_data[(client_id, visitor_id)] = {
            'start_time': None,
            'confirm_time': None,
            'last_step': None,
            'steps': [],
            'age': row['clnt_age'],
            'clnt_tenure': f"{int(row['clnt_tenure_yr'])} years "
        }
    
    step = row['process_step']
    timestamp = row['date_time']
    
    if step == 'start':
        client_data[(client_id, visitor_id)]['start_time'] = timestamp
    elif step == 'confirm':
        client_data[(client_id, visitor_id)]['confirm_time'] = timestamp
    else:
        client_data[(client_id, visitor_id)]['steps'].append((step, timestamp))
    
    if step != 'start':
        client_data[(client_id, visitor_id)]['last_step'] = step

# Process the collected data
for (client_id, visitor_id), data in client_data.items():
    start_time = data['start_time']
    confirm_time = data['confirm_time']
    last_step = data['last_step']
    steps = data['steps']
    
    if start_time is not None and confirm_time is not None:
        completed = 'yes'
        duration = round((confirm_time - start_time).total_seconds() / 60.0, 2)
        last_step = 'confirm'
    else:
        completed = 'no'
        if steps:
            duration = round((steps[-1][1] - start_time).total_seconds() / 60.0, 2) if start_time else None
        else:
            duration = None
    
    new_data.append([client_id, visitor_id, completed, duration, data['age'], data['clnt_tenure'], last_step])

# Create new DataFrame
control_df = pd.DataFrame(new_data, columns=['client_id', 'visitor_id', 'completed', 'duration', 'age', 'clnt_tenure', 'last_step'])

# Add the extra column for completed
control_df['completed_numeric'] = control_df['completed'].map({'yes': 1, 'no': 0})

# Display the resulting DataFrame
control_df

/var/folders/f6/3j2ss2s17kng3g2xrm_hz4sm0000gn/T/ipykernel_35308/3294716554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_control['date_time'] = pd.to_datetime(df_control['date_time'])


,client_id,visitor_id,completed,duration,age,clnt_tenure,last_step,completed_numeric
0,1028,42237450_62128060588,no,8.97,36.0,12 years,step_1,0
1,1104,194240915_18158000533,no,NaN,48.0,5 years,None,0
2,1186,446844663_31615102958,no,0.37,22.0,8 years,step_2,0
3,1195,766842522_69992551638,yes,4.08,54.5,21 years,confirm,1
4,1197,753759429_54481946928,yes,1.58,28.5,7 years,confirm,1
...,...,...,...,...,...,...,...,...
26346,9997391,494669706_3354361161,no,3.72,60.0,4 years,step_3,0
26347,9997470,395791369_55562604618,yes,1.03,36.0,6 years,confirm,1
26348,9997470,91394485_75296404278,no,NaN,36.0,6 years,None,0
26349,9998346,292425655_16607136645,yes,12.47,61.5,50 years,confirm,1


In [ ]:
df_test = final_v1[final_v1['Variation'] == 'Test']
df_test

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417646,1574008,117364417_77840596075,528720790_71583064618_169151,confirm,2017-05-06 23:51:05,Test,10.0,121.0,55.0,U,2.0,153238.83,3.0,6.0
417647,1574008,117364417_77840596075,528720790_71583064618_169151,step_3,2017-05-06 23:50:40,Test,10.0,121.0,55.0,U,2.0,153238.83,3.0,6.0
417648,1574008,117364417_77840596075,528720790_71583064618_169151,step_2,2017-05-06 23:43:47,Test,10.0,121.0,55.0,U,2.0,153238.83,3.0,6.0
417649,1574008,117364417_77840596075,528720790_71583064618_169151,step_1,2017-05-06 23:43:34,Test,10.0,121.0,55.0,U,2.0,153238.83,3.0,6.0


In [ ]:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

# Sort the DataFrame by client_id, visitor_id, and date_time
df_test = df_test.sort_values(by=['client_id', 'visitor_id', 'date_time'])

# Initialize lists to store new dataframe data
new_data = []

# Dictionary to store start times, end times, and last step
client_data = {}

# Iterate over rows to collect start and confirm times, last step, and calculate other needed data
for index, row in df_test.iterrows():
    client_id = row['client_id']
    visitor_id = row['visitor_id']
    
    if (client_id, visitor_id) not in client_data:
        client_data[(client_id, visitor_id)] = {
            'start_time': None,
            'confirm_time': None,
            'last_step': None,
            'steps': [],
            'age': row['clnt_age'],
            'clnt_tenure': f"{int(row['clnt_tenure_yr'])} years "
        }
    
    step = row['process_step']
    timestamp = row['date_time']
    
    if step == 'start':
        client_data[(client_id, visitor_id)]['start_time'] = timestamp
    elif step == 'confirm':
        client_data[(client_id, visitor_id)]['confirm_time'] = timestamp
    else:
        client_data[(client_id, visitor_id)]['steps'].append((step, timestamp))
    
    if step != 'start':
        client_data[(client_id, visitor_id)]['last_step'] = step

# Process the collected data
for (client_id, visitor_id), data in client_data.items():
    start_time = data['start_time']
    confirm_time = data['confirm_time']
    last_step = data['last_step']
    steps = data['steps']
    
    if start_time is not None and confirm_time is not None:
        completed = 'yes'
        duration = round((confirm_time - start_time).total_seconds() / 60.0, 2)
        last_step = 'confirm'
    else:
        completed = 'no'
        if steps:
            duration = round((steps[-1][1] - start_time).total_seconds() / 60.0, 2) if start_time else None
        else:
            duration = None
    
    new_data.append([client_id, visitor_id, completed, duration, data['age'], data['clnt_tenure'], last_step])

# Create new DataFrame
test_df = pd.DataFrame(new_data, columns=['client_id', 'visitor_id', 'completed', 'duration', 'age', 'clnt_tenure', 'last_step'])
# Add the extra column for completed
test_df['completed_numeric'] = test_df['completed'].map({'yes': 1, 'no': 0})

test_df

/var/folders/f6/3j2ss2s17kng3g2xrm_hz4sm0000gn/T/ipykernel_35308/592828030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['date_time'] = pd.to_datetime(df_test['date_time'])


,client_id,visitor_id,completed,duration,age,clnt_tenure,last_step,completed_numeric
0,555,402506806_56087378777,yes,2.63,29.5,3 years,confirm,1
1,647,66758770_53988066587,yes,6.28,57.5,12 years,confirm,1
2,934,810392784_45004760546,no,NaN,51.0,9 years,None,0
3,1336,920624746_32603333901,yes,137.80,42.0,48 years,confirm,1
4,1346,123474046_4204671056,no,0.05,46.0,14 years,step_3,0
...,...,...,...,...,...,...,...,...
30000,9999626,52633065_71189986073,no,0.13,35.0,9 years,step_1,0
30001,9999729,604429154_69247391147,no,0.25,31.0,10 years,step_1,0
30002,9999729,834634258_21862004160,yes,1.25,31.0,10 years,confirm,1
30003,9999729,843385170_36953471821,no,-1.35,31.0,10 years,step_2,0


H0 = Control group success rate = Test group success rate 

H1= Control group success rate != Test group success rate

alpha= 0.05



In [ ]:

control_numeric_df = control_df['completed_numeric']
control_numeric_df = pd.DataFrame(control_numeric_df)
control_numeric_df.shape

test_numeric_df = test_df['completed_numeric']
test_numeric_df = pd.DataFrame(test_numeric_df)
test_numeric_df.shape

(30005, 1)

In [ ]:
t_stat, p_val = ttest_ind(control_numeric_df['completed_numeric'], test_numeric_df['completed_numeric'], equal_var=False)
print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_val}")

T-Statistic: -8.130156856984316
P-Value: 4.3755383431637894e-16


Interpretation - 

H0 = Control group success rate > Test group success rate 

H1= Control group success rate < Test group success rate

In [ ]:
t_stat, p_val = stats.ttest_ind(test_numeric_df, control_numeric_df, alternative='greater')

print(f"T-Statistic: {t_stat}")
print(f"P-Value: {p_val}")

T-Statistic: [8.1373242]
P-Value: [2.06140469e-16]


Interpretation of the Results
T-Statistic: Indicates how many standard deviations the test group mean is from the control group mean.
P-Value: The probability of observing the data if the null hypothesis is true. A lower p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis.
If the p-value is less than or equal to your significance level (e.g., 0.05), you reject the null hypothesis and conclude that the mean success rate of the test group is significantly greater than that of the control group. If the p-value is greater than the significance level, you fail to reject the null hypothesis and conclude that there is not enough evidence to say the mean success rate of the test group is greater than that of the control group.

H0 = Success rate is more when for age group< 47
H1 = Success rate is more when for age group> 47

In [ ]:
Q1  = test_df['age'].quantile(.25)
Q3  = test_df['age'].quantile(.75)
IQR = Q3- Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = test_df[(test_df['age'] >= lower_bound) & (test_df['age'] <= upper_bound)]
df_filtered

,client_id,visitor_id,completed,duration,age,clnt_tenure,last_step,completed_numeric
0,555,402506806_56087378777,yes,2.63,29.5,3 years,confirm,1
1,647,66758770_53988066587,yes,6.28,57.5,12 years,confirm,1
2,934,810392784_45004760546,no,NaN,51.0,9 years,None,0
3,1336,920624746_32603333901,yes,137.80,42.0,48 years,confirm,1
4,1346,123474046_4204671056,no,0.05,46.0,14 years,step_3,0
...,...,...,...,...,...,...,...,...
30000,9999626,52633065_71189986073,no,0.13,35.0,9 years,step_1,0
30001,9999729,604429154_69247391147,no,0.25,31.0,10 years,step_1,0
30002,9999729,834634258_21862004160,yes,1.25,31.0,10 years,confirm,1
30003,9999729,843385170_36953471821,no,-1.35,31.0,10 years,step_2,0


In [ ]:
df_filtered['age'].mean().round(2)

47.65

In [ ]:
group_1 = df_filtered[df_filtered['age'] <= 47]['completed_numeric']  # Aged 47 or younger
group_2 = df_filtered[df_filtered['age'] > 47]['completed_numeric']    # Aged over 47

# Step 2: Calculate success rates
success_1 = group_1.sum()  # Successes in group 1
total_1 = group_1.count()   # Total in group 1

success_2 = group_2.sum()  # Successes in group 2
total_2 = group_2.count()   # Total in group 2

# Step 3: Perform a two-sample proportion test
successes = [success_1, success_2]
totals = [total_1, total_2]
z_stat, p_value = proportions_ztest(successes, totals)
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is evidence that age > 47 has a higher success rate.")
else:
    print("Fail to reject the null hypothesis: No evidence that age > 47 has a higher success rate.")

# Output results
print(f"Z-statistic: {z_stat}, P-value: {p_value}")
print(f"Success rate for age <= 47: {success_1 / total_1:.2%} (n={total_1})")
print(f"Success rate for age > 47: {success_2 / total_2:.2%} (n={total_2})")

Reject the null hypothesis: There is evidence that age > 47 has a higher success rate.
Z-statistic: 12.8342985792573, P-value: 1.0535279360743302e-37
Success rate for age <= 47: 65.16% (n=14314)
Success rate for age > 47: 57.94% (n=15691)


In [ ]:
Q1  = control_df['age'].quantile(.25)
Q3  = control_df['age'].quantile(.75)
IQR = Q3- Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered_control = control_df[(control_df['age'] >= lower_bound) & (control_df['age'] <= upper_bound)]
df_filtered_control

,client_id,visitor_id,completed,duration,age,clnt_tenure,last_step,completed_numeric
0,1028,42237450_62128060588,no,8.97,36.0,12 years,step_1,0
1,1104,194240915_18158000533,no,NaN,48.0,5 years,None,0
2,1186,446844663_31615102958,no,0.37,22.0,8 years,step_2,0
3,1195,766842522_69992551638,yes,4.08,54.5,21 years,confirm,1
4,1197,753759429_54481946928,yes,1.58,28.5,7 years,confirm,1
...,...,...,...,...,...,...,...,...
26346,9997391,494669706_3354361161,no,3.72,60.0,4 years,step_3,0
26347,9997470,395791369_55562604618,yes,1.03,36.0,6 years,confirm,1
26348,9997470,91394485_75296404278,no,NaN,36.0,6 years,None,0
26349,9998346,292425655_16607136645,yes,12.47,61.5,50 years,confirm,1


In [ ]:
df_filtered_control['age'].mean()

47.80672460248188

In [ ]:
group_1 = df_filtered_control[df_filtered_control['age'] <= 47]['completed_numeric']  # Aged 47 or younger
group_2 = df_filtered_control[df_filtered_control['age'] > 47]['completed_numeric']    # Aged over 47

# Step 2: Calculate success rates
success_1 = group_1.sum()  # Successes in group 1
total_1 = group_1.count()   # Total in group 1

success_2 = group_2.sum()  # Successes in group 2
total_2 = group_2.count()   # Total in group 2

# Step 3: Perform a two-sample proportion test
successes = [success_1, success_2]
totals = [total_1, total_2]
z_stat, p_value = proportions_ztest(successes, totals)
alpha = 0.05  # significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is evidence that age > 47 has a higher success rate.")
else:
    print("Fail to reject the null hypothesis: No evidence that age > 47 has a higher success rate.")

# Output results
print(f"Z-statistic: {z_stat}, P-value: {p_value}")
print(f"Success rate for age <= 47: {success_1 / total_1:.2%} (n={total_1})")
print(f"Success rate for age > 47: {success_2 / total_2:.2%} (n={total_2})")

Reject the null hypothesis: There is evidence that age > 47 has a higher success rate.
Z-statistic: 6.863014301546612, P-value: 6.7422473613817095e-12
Success rate for age <= 47: 60.21% (n=12492)
Success rate for age > 47: 56.04% (n=13859)


H0 = 
H1 = Longer the duration more the success rate.

In [ ]:
control_df['completed_numeric'].unique()

array([0, 1])

In [ ]:
correlation=test_df['duration'].corr(test_df['completed_numeric'])
correlation

0.12921683072072998

Insight- A positive correlation coefficient suggests that longer durations are associated with higher success rates.
The p-value indicates whether this correlation is statistically significant (commonly, a p-value < 0.05 is considered significant)

In [ ]:
df_filtered_control['duration'].mean()

-117.87910049423395

Completion Rate: The proportion of users who reach the final ‘confirm’ step.

In [ ]:
['completed_numeric']

In [ ]:
# Convert the 'date_time' column to datetime format
merged_df2['date_time'] = pd.to_datetime(merged_df2['date_time'])

# Group by specified columns and aggregate the 'date_time' into lists
grouped_df = merged_df2.groupby(['client_id', 'Variation', 'visitor_id', 'visit_id', 'process_step'])['date_time'].agg(list).reset_index()

# Pivot the DataFrame to make process_step values into columns
pivot_df = grouped_df.pivot_table(
    index=['client_id', 'Variation', 'visitor_id', 'visit_id'],  # Use these columns as the index
    columns='process_step',  # Pivot on process_step
    values='date_time',      # Values to fill the new DataFrame with
    aggfunc=lambda x: x      # Keep all occurrences
)

# Reset index to turn the multi-index into regular columns
pivot_df.reset_index(inplace=True)